In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import wntr
from pathlib import Path
import wntr.network.controls as controls
from dreaminsg_integrated_model.src.network_sim_models.interdependencies import *
from dreaminsg_integrated_model.src.network_recovery import *
import pandapower as pp

import math
import pandapower.plotting as pandaplot
import copy

In [3]:
network_dir= 'micropolis'

MAIN_DIR = Path('../..')
water_file = MAIN_DIR/'dreaminsg_integrated_model/data/networks/{0}/water/water.inp'.format(network_dir)

step_size = 60
try:
    wn_original = wntr.network.WaterNetworkModel(water_file)
    wn_original.options.time.duration = 3600
    wn_original.options.time.report_timestep = 3600
    wn_original.options.time.hydraulic_timestep = 60
    wn_original.options.hydraulic.demand_model = 'DDA'
    
    print("Water network successfully loaded.")
except FileNotFoundError:
    print("Error: The water network file does not exist. No such file or directory: ", water_file)

Water network successfully loaded.


C:\Users\srijith\anaconda3\envs\redcar\lib\site-packages\wntr\epanet\io.py:2173: UserWarning: Not all curves were used in "..\..\dreaminsg_integrated_model\data\networks\micropolis\water\water.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


In [10]:
wn = copy.deepcopy(wn_original)
components = ['W_PMA1050', 'W_PMA1118', 'W_PMA609', 'W_PMA608' , 'W_PMA937']
start_time = 6000
end_time = 22000

for component in components:
    wn = wntr.morph.split_pipe(wn,component,f"{component}_B",f"{component}_leak_node",)

    leak_node = wn.get_node(f"{component}_leak_node")
    leak_node.remove_leak(wn)
    leak_node.add_leak(wn,area=0.75*(math.pi*(wn.get_link(f"{component}_B").diameter)** 2)/4,
        start_time=start_time,
        end_time=end_time,
    )

    link_close_event(wn, component, end_time, "repairing")
    link_open_event(wn, component, end_time + 20000, "repairing")

In [11]:
while wn.options.time.duration <=  3600*48:
    print("Simulation time: ", wn.options.time.duration,"; Hydraulic time step: ", wn.options.time.hydraulic_timestep, "; Report time step: ", wn.options.time.report_timestep)
    wn_sim = wntr.sim.WNTRSimulator(wn)
    wn_results = wn_sim.run_sim(convergence_error=True, solver_options={"MAXITER": 10000})

    print('failed pipe flowrate: ', wn_results.link['flowrate'][components].values)
    print('total leak demand: ', wn_results.node['leak_demand'][[f'{component}_leak_node' for component in components]].values)
    print('tank head: ', wn_results.node['head']['W_T1'].values)
    print('total pump flow rate: ', wn_results.link['flowrate'][wn.pump_name_list].values)
    print('***************')
    wn.options.time.duration += 3600

Simulation time:  3600.0 ; Hydraulic time step:  60.0 ; Report time step:  3600
failed pipe flowrate:  [[-0.001 -0.003 -0.004  0.002 -0.001]
 [-0.001 -0.003 -0.004  0.002 -0.001]]
total leak demand:  [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
tank head:  [352.044 352.681]
total pump flow rate:  [[0.001 0.001 0.001 0.001 0.068 0.    0.    0.064]
 [0.001 0.001 0.001 0.001 0.068 0.    0.    0.064]]
***************
Simulation time:  7200.0 ; Hydraulic time step:  60.0 ; Report time step:  3600
failed pipe flowrate:  [[ 0.009  0.015  0.012 -0.006  0.018]]
total leak demand:  [[0.007 0.012 0.071 0.003 0.023]]
tank head:  [352.573]
total pump flow rate:  [[0.002 0.002 0.002 0.002 0.074 0.    0.    0.064]]
***************
Simulation time:  10800.0 ; Hydraulic time step:  60.0 ; Report time step:  3600
failed pipe flowrate:  [[ 0.009  0.015  0.014 -0.006  0.017]]
total leak demand:  [[0.006 0.01  0.07  0.003 0.022]]
tank head:  [351.021]
total pump flow rate:  [[0.003 0.003 0.003 0.003 0.074 0.    0.

In [ ]:
wn_results.node['demand'].to_csv('results.csv')

In [ ]:
stop

In [ ]:
t = 86400
node_results = wn_results.node['demand'].iloc[-1]

water_supplied = sum([node_results[junc] for junc in wn.junction_name_list])

base_demands_at_t = []

for junc in wn.junction_name_list:
    base_demand = wn.get_node(junc).base_demand
    
    if base_demand != 0:
        demand = node_results[junc]
        pattern = wn.get_node(junc).demand_timeseries_list[0].pattern.multipliers
        #print(pattern)
        pattern_size = len(pattern)
        pattern_interval = 24/pattern_size
        
        pattern_index = math.ceil(((t/3600)%86400) / pattern_interval)
        multiplier = pattern[pattern_index-1]
        base_demands_at_t.append(multiplier*base_demand)
        

water_resmetric = water_supplied/sum(base_demands_at_t)
print(water_resmetric)

1.0


In [ ]:
wn.pump_name_list

['W_WP1', 'W_WP2', 'W_WP3', 'W_WP4', 'W_WP5', 'W_WP6', 'W_WP7', 'W_WP8']